In [8]:
import pandas as pd

In [9]:
METEO_DTYPES = {'temperature':'float64', 'relative_humidity':'float64','precipitation_rate':'float64', 
                'wind_speed':'float64','zipcode':'str'}

In [15]:
df_meteo = pd.read_csv('/Users/Jakob_1/Desktop/2024_EAE/DMBI/0_Final Assignment/final_assignment_dmbi/data/meteo_eae.csv', 
                       delimiter=';',dtype = METEO_DTYPES, parse_dates= ['date'])

In [19]:
df_meteo

,date,temperature,relative_humidity,precipitation_rate,wind_speed,zipcode
0,2024-01-01,9.23763,80.46236,0.00001,0.86917,00005
1,2024-01-01,2.85132,70.34123,0.00000,1.19543,00040
2,2024-01-01,12.52156,68.36415,0.00000,1.74316,00041
3,2024-01-01,2.97691,74.22747,0.00000,3.94315,00042
4,2024-01-01,8.62595,54.41100,0.00000,3.17850,00043
...,...,...,...,...,...,...
4114201,2024-12-31,10.12473,68.43024,0.00000,2.61134,80154
4114202,2024-12-31,10.26732,57.08380,0.00000,1.28835,80338
4114203,2024-12-31,5.99634,59.47399,0.00000,0.71365,90007
4114204,2024-12-31,7.18663,67.74951,0.00000,1.54551,91319
